**The Environment**

The environment for this project is a graph of nodes connected by edges. The agent, the prey, and the predator,
can move between the nodes along the edges. There are 50 nodes, numbered 1 to 50, connected in a large circle.

Additionally, add edges at random to increase connectivity across the circle, in the following way:

 Picking a random node with degree less than 3,
 add an edge between it and one node within 5 steps forward or backward along the primary loop. (So node 10
might get connected to node 7 or node 15, but not node 16.)
 Do this until no more edges can be added.

Graph Theory Question: With this setup, you can add at most 25 additional edges (why?). Are you
always able to add this many? If not, what's the smallest number of edges you're always able to add?*

*Creating circle loop grapgh*

In [ ]:
confidence=[]
confidence_pred = []
#Global lists to be used later

In [ ]:
def creategraph():
    graph = {}

    nodes = 50
    for node in range(1,nodes+1):
        graph[node] = set()

    for node in range(1,nodes):
        if node != 1:
            graph[node].add(node-1)
        
        graph[node].add(node+1)
        #graph[node+1].add(node)
        if node == 49:
            graph[50].add(node)
            graph[50].add(1)
            graph[1].add(50)
    #print("The circle loop graph generates is:\n",graph)
    return graph
graph = creategraph()


*Adding extra edges to the graph*

In [ ]:
import random

#degree_reached = False
#while degree_reached == False:
def add_edges_to_graph(graph):
    old_dic = {}
    while old_dic != graph:
        old_dic = graph
        for node in range(1,51):
            #print(graph[node])
            if len(graph[node]) < 3:
                #print("aya")
                if random.choice([1,0]) == 1: #* if 1 the add edge
                    add_to_node = node + random.choice([random.randint(1,6),random.randint(-5,-1)])
                    #print("add to node: ", add_to_node)
                    if add_to_node < 1:
                        add_to_node = add_to_node + 50 
                    elif add_to_node > 50:
                            add_to_node = add_to_node - 50
                    if len(graph[add_to_node]) < 3:
                        graph[node].add(add_to_node)
                        graph[add_to_node].add(node)
                        
                #else:
            #    degree_reached = True

    #print(graph)
    return graph
graph = add_edges_to_graph(graph)


Generating Multiple Mazes

In [ ]:
# import pandas as pd

def gen_100_graphs():
    Graph100 = {}
    for num in range(100):
        graph = creategraph()
        graph = add_edges_to_graph(graph)
        Graph100[num] = graph
    
    return Graph100
    
Graph100 = gen_100_graphs()
graph = Graph100[0] #* we are using this graph
print(graph)
#print(Graph100)


Visualising the graph created

In [ ]:
# import numpy as np
# import networkx as nx
# from pyvis.network import Network


# G = nx.from_dict_of_lists(graph, create_using=nx.MultiDiGraph)
# G2 = Network()
# G2.from_nx(G)
# G2.show('network_map.html')

Prey and Predator

Retraced shortest path

In [ ]:
def retraced_path(prev,curr_node,goal): #* changed prev in func def
    #print("entering retracd path") #* no need to print
    fwd_path = {}
    #fwd_stack = LifoQueue()
    cell = goal #4,4
    
    while prev[cell]!= None and cell != curr_node  :
        
        fwd_path[prev[cell]] = cell
        #fwd_stack.put((cell,prev[cell]))
        cell = prev[cell]
        
   
     #* return the path to be followed by the agent in the order from starting to end
    return len(fwd_path.keys())

Spawning Agent, prey, predator

In [ ]:
Positions = {"Agent": None,"Prey":None,"Pred":None}

def spawn(graph,Positions):
    Agent_pos = random.choice(list(graph.keys()))
    Positions["Agent"] = Agent_pos
    while Positions["Prey"] == None and Positions["Pred"] == None:
        prey_pos = random.choice(list(graph.keys()))
        pred_pos = random.choice(list(graph.keys()))

        if Positions["Prey"] == None:
            Positions["Prey"] = prey_pos
        if Positions["Pred"] == None:
            Positions["Pred"] = pred_pos
    return Positions

Positions = spawn(graph,Positions)
    


In [ ]:
import numpy as np
import random
import math

from queue import Queue 

def bfs(curr_pred_node,curr_agent_node):
    goal = curr_agent_node
    queue = Queue()
    prev = {
        curr_pred_node: None
    }
    visited = set()
    queue.put(curr_pred_node)
    #explore = []
    while queue.empty() == False:
        curr_state = queue.get()
        
        
        if curr_state not in visited:
           # explore.append(curr_state)
        
            if curr_state == goal:
                #print("found the path for agent 1") #, curr_state,prev) # #* no need to print
                #print(explore) #* no need to print
                shortest_path = retraced_path(prev,curr_pred_node,goal)
                return shortest_path
                #break
            else:
                for child in graph[curr_state]:
                    if child not in visited : 
                    
                        queue.put(child)
                        if child not in prev.keys():# --> check this
                            prev[child] = curr_state
                visited.add(curr_state)
    
    return 




def move_prey(curr_prey_node):
    prey_neighbors = [node for node in graph[curr_prey_node]]
    prey_neighbors.append(curr_prey_node)
    selected_prey_neighbor = random.choice(prey_neighbors)
    #print(selected_prey_neighbor)
    return(selected_prey_neighbor)
def move_predator(curr_pred_node,curr_agent_node,prob_req = False): #* added prob req for agent 5 to get the denominator of probability
    min_dist_to_agent = math.inf
    breaking_ties_list = []
    for neighbor in graph[curr_pred_node]:
        dist_to_agent = bfs(neighbor,curr_agent_node)
        if dist_to_agent <= min_dist_to_agent:
            min_dist_to_agent = dist_to_agent
            #selected_pred_neighbor = neighbor
            breaking_ties_list.append(neighbor)
    if prob_req == True:    #* added prob req for agent 5 to get the denominator of probability
        return breaking_ties_list
                
        #print(breaking_ties_list)
    selected_pred_neighbor = random.choice(breaking_ties_list)

    
    return(selected_pred_neighbor)

   # print("build function")

print("prey child chosen:" ,move_prey(5))
print("pred child chosen:",move_predator(3,49))
print(graph)
print("hello")

Adding Easily Distracted Predator

In [ ]:
def move_distracted_predator(curr_pred_node,curr_agent_node,prob_req = False): #* added prob req for agent 5 to get the denominator of probability
    min_dist_to_agent = math.inf
    breaking_ties_list = []
    for neighbor in graph[curr_pred_node]:
        dist_to_agent = bfs(neighbor,curr_agent_node)
        if dist_to_agent <= min_dist_to_agent:
            min_dist_to_agent = dist_to_agent
            #selected_pred_neighbor = neighbor
            breaking_ties_list.append(neighbor)
    if prob_req == True:    #* added prob req for agent 5 to get the denominator of probability
        return breaking_ties_list
                
        #print(breaking_ties_list)
    pred_neighbour_close = random.choice(breaking_ties_list) #* this points to the 0.6 prob part
    #print(graph[curr_pred_node]) #* del this later
    pred_neighbour_all = random.choice(list(graph[curr_pred_node])) #* this points to the 0.4 prob part
    selected_pred_neighbor = random.choices([pred_neighbour_close,pred_neighbour_all],weights = [0.6,0.4])[0]

    
    return(selected_pred_neighbor)

Agent 1

In [ ]:
def print_pos(agent1,prey1,pred1):
        # print("Agent", agent1)
        # print("prey", prey1)
        # print("Pred", pred1)
        pass

def step_count(steps):
    steps = steps+1
    return steps

def agent_one(agent_pos, prey_pos,pred_pos, maxsteps):

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps:
            return "Faliure2"
        #distances of agent from prey and pred
        x = bfs(agent_pos, prey_pos)
        y = bfs(agent_pos, pred_pos)
        cost_dic = {}

        for neighbour in graph[agent_pos]:
            
            cost_dic[neighbour] = [bfs(neighbour, prey_pos)]
            cost_dic[neighbour].append(bfs(neighbour,pred_pos))

        #Now checking conditions

        green = [ ]
        for neighbour in cost_dic:
            if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] > y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue

        for neighbour in cost_dic:
            if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] == y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue

        for neighbour in cost_dic:
            if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] > y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue

        for neighbour in cost_dic:
            if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] == y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue

        for neighbour in cost_dic:
            if cost_dic[neighbour][1] > y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue

        for neighbour in cost_dic:
            if cost_dic[neighbour][1] == y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue
        
        else:
            prey_pos = move_prey(prey_pos)
            pred_pos = move_predator(pred_pos, agent_pos)
            print_pos(agent_pos,prey_pos,pred_pos)
            continue
    
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"

AGENT 2

In [ ]:
def agent_two_remix(agent_pos, prey_pos,pred_pos,maxsteps):

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps:
            return "Faliure2"
        #distances of agent from prey and pred
        x = bfs(agent_pos, prey_pos)
        y = bfs(agent_pos, pred_pos)
        cost_dic = {}

        for neighbour in graph[agent_pos]:
            
            cost_dic[neighbour] = [bfs(neighbour, prey_pos)]
            cost_dic[neighbour].append(bfs(neighbour,pred_pos))

        #Now checking conditions

        green = [ ]

        if y <=5:
            
            
            for neighbour in cost_dic:
                if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] > y:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue

            for neighbour in cost_dic:
                if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] == y:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue

            for neighbour in cost_dic:
                if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] > y:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue

            for neighbour in cost_dic:
                if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] == y:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue

            for neighbour in cost_dic:
                if cost_dic[neighbour][1] > y:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue

            for neighbour in cost_dic:
                if cost_dic[neighbour][1] == y:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue
            
            else:
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
                continue

        else:
            
            for neighbour in cost_dic:
                if cost_dic[neighbour][0] < x:
                    green.append(neighbour)
            if len(green)!=0:
                agent_pos =  random.choice(green)
                prey_pos = move_prey(prey_pos)
                pred_pos = move_predator(pred_pos, agent_pos)
                print_pos(agent_pos,prey_pos,pred_pos)
    
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"

Agent one rules

In [ ]:
def agent_one_rules(agent_pos,prey_pos,pred_pos):

    x = bfs(agent_pos, prey_pos)
    y = bfs(agent_pos, pred_pos)
    cost_dic = {}

    for neighbour in graph[agent_pos]:
        
        cost_dic[neighbour] = [bfs(neighbour, prey_pos)]
        cost_dic[neighbour].append(bfs(neighbour,pred_pos))

    #Now checking conditions

    green = [ ]
    for neighbour in cost_dic:
        if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] > y:
            green.append(neighbour)
    if len(green)!=0:
        agent_pos =  random.choice(green)
        return agent_pos

    for neighbour in cost_dic:
        if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] == y:
            green.append(neighbour)
    if len(green)!=0:
        agent_pos =  random.choice(green)
        return agent_pos

    for neighbour in cost_dic:
        if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] > y:
            green.append(neighbour)
    if len(green)!=0:
        agent_pos =  random.choice(green)
        return agent_pos

    for neighbour in cost_dic:
        if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] == y:
            green.append(neighbour)
    if len(green)!=0:
        agent_pos =  random.choice(green)
        return agent_pos

    for neighbour in cost_dic:
        if cost_dic[neighbour][1] > y:
            green.append(neighbour)
    if len(green)!=0:
        agent_pos =  random.choice(green)
        return agent_pos

    for neighbour in cost_dic:
        if cost_dic[neighbour][1] == y:
            green.append(neighbour)
    if len(green)!=0:
        agent_pos =  random.choice(green)
        return agent_pos
        
    else:
        return agent_pos


Agent two rules

In [ ]:
def agent_two_rules(agent_pos,prey_pos,pred_pos):

    x = bfs(agent_pos, prey_pos)
    y = bfs(agent_pos, pred_pos)
    cost_dic = {}

    for neighbour in graph[agent_pos]:
        
        cost_dic[neighbour] = [bfs(neighbour, prey_pos)]
        cost_dic[neighbour].append(bfs(neighbour,pred_pos))

    #Now checking conditions

    green = [ ]


    if y <=5:
            
            
        for neighbour in cost_dic:
            if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] > y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
            

        for neighbour in cost_dic:
            if cost_dic[neighbour][0] < x and cost_dic[neighbour][1] == y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
            

        for neighbour in cost_dic:
            if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] > y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
            

        for neighbour in cost_dic:
            if cost_dic[neighbour][0] == x and cost_dic[neighbour][1] == y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
            

        for neighbour in cost_dic:
            if cost_dic[neighbour][1] > y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
            

        for neighbour in cost_dic:
            if cost_dic[neighbour][1] == y:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
        
        else:
            return agent_pos
                

    else:
            
        for neighbour in cost_dic:
            if cost_dic[neighbour][0] < x:
                green.append(neighbour)
        if len(green)!=0:
            agent_pos =  random.choice(green)
            return agent_pos
    
    return agent_pos
        



Functions for agent 3 and 4


In [ ]:
def prey_prob_initial(graph,agent_pos):
    prob_dic = {}
    for nodes in graph:
        if nodes == agent_pos:
            prob_dic[nodes] = 0
        else:
            prob_dic[nodes] = 1/49
    return prob_dic

def prey_prob_updates_Agent(prob_dic,prey_found,surveyed_node):
    prob_dic_update = {}
    for nodes in prob_dic:
        if prey_found == False: #* Case 1
            if nodes != surveyed_node:
                prob_dic_update[nodes] = prob_dic[nodes] / (1 - prob_dic[surveyed_node])
            else:
                prob_dic_update[nodes] = 0
        elif prey_found == True: #* Case 2
            if nodes != surveyed_node:
                prob_dic_update[nodes] = 0
            else:
                prob_dic_update[nodes] = 1
    
    #prob_dic = prob_dic_update
    return prob_dic_update

def prey_prob_updates_prey(prob_dic,graph): #* When prey moves, ie. t+1
    prob_dic_update = {}
    for nodes in prob_dic:
        prob_update = 0
        for sub_nodes in prob_dic:
            if sub_nodes == nodes:
                prob_update +=  prob_dic[sub_nodes] * (1/(len(graph[sub_nodes])+1)) # was 0
            else:
                if nodes in graph[sub_nodes]:
                    prob_update += prob_dic[sub_nodes] * (1/(len(graph[sub_nodes])+1))
                else:
                    prob_update += 0
        prob_dic_update[nodes] = prob_update
    
    return prob_dic_update




Agent 3

In [ ]:
def node_survey(node, prey_pos):
    if node == prey_pos:
        return(True)
    else:
        return(False)


def agent_three(agent_pos, prey_pos,pred_pos,maxsteps):
    
    prob_dic = prey_prob_initial(graph,agent_pos)
    

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        result = node_survey(survey_node,prey_pos)
        if result == True:
            confidence.append(True)
        else:
            confidence.append(False)

        #Update probability at time t with new info
        prob_dic = prey_prob_updates_Agent(prob_dic,result,survey_node)

        #getting the assumed prey postion
        pseudo_prey_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        
        #agent moves
        agent_pos= agent_one_rules(agent_pos, pseudo_prey_pos, pred_pos)
        Result =  (agent_pos == prey_pos)
        if Result == True:
            return "Success"
        #Update probability at time t with new info
        prob_dic = prey_prob_updates_Agent(prob_dic,Result,agent_pos)
        
        
        prey_pos = move_prey(prey_pos)

        #Update probability at time t+1
        prob_dic = prey_prob_updates_prey(prob_dic,graph)
        

        pred_pos = move_predator(pred_pos, agent_pos) 
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"

Agent 4

In [ ]:
def agent_four(agent_pos, prey_pos,pred_pos,maxsteps):

    #Initializing dictionary 
    # prob_dic = {}
    # for i in range(1,51):
    #     prob_dic[i] = 1/49

    prob_dic = prey_prob_initial(graph,agent_pos)
    

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        result = node_survey(survey_node,prey_pos)

        if result == True:
            confidence.append(True)
        else:
            confidence.append(False)

        #Update probability at time t with new info
        prob_dic = prey_prob_updates_Agent(prob_dic,result,survey_node)

        #getting the assumed prey postion
        pseudo_prey_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        
        #agent moves
        agent_pos= agent_two_rules(agent_pos, pseudo_prey_pos, pred_pos)
        Result =  (agent_pos == prey_pos)
        if Result == True:
            return "Success"
            
        #Update probability at time t with new info
        prob_dic = prey_prob_updates_Agent(prob_dic,Result,agent_pos)
        
        
        prey_pos = move_prey(prey_pos)

        #Update probability at time t+1
        prob_dic = prey_prob_updates_prey(prob_dic,graph)

        pred_pos = move_predator(pred_pos, agent_pos) 
         
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Functions for agent 5 and 6

In [ ]:
def pred_prob_initial(graph,agent_pos,pred_start): #* pred start is the pred initial position (Writeup currently mentions that agent knows pred's start position)
    prob_dic = {}
    for nodes in graph:
        if nodes == agent_pos:
            prob_dic[nodes] = 0
        elif nodes == pred_start:
            prob_dic[nodes] = 1
        else:
            prob_dic[nodes] = 0
    #print("Beleif sum initial: ",sum(prob_dic.values())) #* del this later
    return prob_dic

def pred_prob_updates_Agent(prob_dic,pred_found,surveyed_node):
    prob_dic_update = {}
    for nodes in prob_dic:
        if pred_found == False: #* Case 1
            if nodes != surveyed_node:
                prob_dic_update[nodes] = prob_dic[nodes] / (1 - prob_dic[surveyed_node])
            else:
                prob_dic_update[nodes] = 0
        elif pred_found == True: #* Case 2
            if nodes != surveyed_node:
                prob_dic_update[nodes] = 0
            else:
                prob_dic_update[nodes] = 1
    
    #prob_dic = prob_dic_update
    #print("Beleif sum survey/move: ",sum(prob_dic.values())) #* del this later
    return prob_dic_update

def pred_prob_updates_pred(prob_dic,graph,agent_pos): #* When pred moves, ie. t+1
    prob_dic_update = {}
    for nodes in prob_dic:
        prob_update = 0
        for sub_nodes in prob_dic:
            if sub_nodes == nodes:
                #prob_update +=  prob_dic[sub_nodes] * (1/(len(graph[sub_nodes])+1)) # was 0
                prob_update += 0
            else:
                if nodes in graph[sub_nodes]:
                    num_options_pred = move_distracted_predator(sub_nodes,agent_pos,True) #* prob req = True for getting prob denominator from move_predator func
                    if nodes in num_options_pred:
                        #if random.choices([0.6,0.4],[0.6,0.4])[0] == 0.6:
                        prob_update += prob_dic[sub_nodes] * ((0.6 / len(num_options_pred)) + (0.4 / len(graph[sub_nodes])))  #(random.choices([0.6,0.4],[0.6,0.4])[0]/len(num_options_pred)) #* check this changed
                        # else:
                        #     prob_update += prob_dic[sub_nodes] * (0.4 / len(num_options_pred))

                    else:
                        prob_update+= prob_dic[sub_nodes] * (0.4 / len(graph[sub_nodes]))
                    #prob_update += prob_dic[sub_nodes] * (1/(len(graph[sub_nodes])+1))
                else:
                    prob_update += 0
        prob_dic_update[nodes] = prob_update
    #print("Beleif sum pred move: ",sum(prob_dic.values())) #* del this later
    return prob_dic_update



Agent 5

In [ ]:
def agent_five(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        result = node_survey(survey_node,pred_pos)
        if result == True:
            confidence.append(True)
        else:
            confidence.append(False)
            
        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,result,survey_node)

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        
        #agent moves
        agent_pos= agent_one_rules(agent_pos, prey_pos, pseudo_pred_pos)
        Result =  (agent_pos == pred_pos)
        if(Result == True):
            return "Faliure1"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Result,agent_pos)
       
        
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos) 

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

#agent_five(14,45,5,1)

Agent 6

In [ ]:
def agent_six(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        result = node_survey(survey_node,pred_pos)
        if result == True:
            confidence.append(True)
        else:
            confidence.append(False)
        
        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,result,survey_node)

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        
        #agent moves
        agent_pos= agent_two_rules(agent_pos, prey_pos, pseudo_pred_pos)
        Result =  (agent_pos == pred_pos)
        if(Result == True):
            return "Faliure1"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Result,agent_pos)
       
        
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos) 

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

#agent_six(14,45,5,100)

Agent 7

In [ ]:
def agent_seven(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
            survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            result_pred = node_survey(survey_node,pred_pos)
            result_prey = node_survey(survey_node,prey_pos)
        else:
            survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
            result_pred = node_survey(survey_node,pred_pos)
            result_prey = node_survey(survey_node,prey_pos)

        if result_prey == True:
            confidence.append(True)
        else:
            confidence.append(False)
               
        if result_pred == True:
            confidence_pred.append(True)
        else:
            confidence_pred.append(False)
        

        #Update probability at time t with new info for both prey and predator
        prob_dic = pred_prob_updates_Agent(prob_dic,result_pred,survey_node)
        prob_dic_prey =  prey_prob_updates_Agent(prob_dic_prey,result_prey,survey_node) # for prey
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])
        
        #agent moves
        agent_pos= agent_one_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"
        Prey_Result =  (agent_pos == prey_pos)
        if(Prey_Result == True):
            return "Success"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
        prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Agent 8

In [ ]:
def agent_eight(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"


        #Surveying a node
        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
            survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            result_pred = node_survey(survey_node,pred_pos)
            result_prey = node_survey(survey_node,prey_pos)
        else:
            survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
            result_pred = node_survey(survey_node,pred_pos)
            result_prey = node_survey(survey_node,prey_pos)
        if result_prey == True:
            confidence.append(True)
        else:
            confidence.append(False)
        if result_pred == True:
            confidence_pred.append(True)
        else:
            confidence_pred.append(False)

        #Update probability at time t with new info for both prey and predator
        prob_dic = pred_prob_updates_Agent(prob_dic,result_pred,survey_node)
        prob_dic_prey =  prey_prob_updates_Agent(prob_dic_prey,result_prey,survey_node) # for prey
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])
        
        #agent moves
        agent_pos= agent_two_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"
        Prey_Result =  (agent_pos == prey_pos)
        if(Prey_Result == True):
            return "Success"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
        prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Agent 7 Modified functions

In [ ]:
#0.9 probability that the result of surveyed node is correct
def prey_prob_updates_Agent_modified(prob_dic,prey_found,surveyed_node):
    prob_dic_update = {}
    for nodes in prob_dic:
        if prey_found == False: #* Case 1
            if nodes != surveyed_node:
                prob_dic_update[nodes] = prob_dic[nodes] / (1 - 0.9*prob_dic[surveyed_node])# prob_dic[nodes] / (1 - prob_dic[surveyed_node])
            else:
                prob_dic_update[nodes] = 0.1*prob_dic[nodes]/(1-0.9*prob_dic[nodes])# 0
        elif prey_found == True: #* Case 2
            if nodes != surveyed_node:
                prob_dic_update[nodes] = 0
            else:
                prob_dic_update[nodes] = 1
    
    #prob_dic = prob_dic_update
    return prob_dic_update

def pred_prob_updates_Agent_modified(prob_dic,pred_found,surveyed_node):
    prob_dic_update = {}
    for nodes in prob_dic:
        if pred_found == False: #* Case 1
            if nodes != surveyed_node:
                prob_dic_update[nodes] = prob_dic[nodes] / (1 - 0.9*prob_dic[surveyed_node])# prob_dic[nodes] / (1 - prob_dic[surveyed_node])
            else:
                prob_dic_update[nodes] = 0.1*prob_dic[nodes]/(1-0.9*prob_dic[nodes]) #0
        elif pred_found == True: #* Case 2
            if nodes != surveyed_node:
                prob_dic_update[nodes] = 0
            else:
                prob_dic_update[nodes] = 1
    
    #prob_dic = prob_dic_update
    #print("Beleif sum survey/move: ",sum(prob_dic.values())) #* del this later
    return prob_dic_update

Agent 7 modified

In [ ]:
def agent_seven_modified(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
            survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
        else:
            survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
            

        #Update probability at time t with new info for both prey and predator
        prob_dic = pred_prob_updates_Agent_modified(prob_dic,result_pred,survey_node)
        prob_dic_prey =  prey_prob_updates_Agent_modified(prob_dic_prey,result_prey,survey_node) # for prey
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])
        
        #agent moves
        agent_pos= agent_one_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"
        Prey_Result =  (agent_pos == prey_pos)
        if(Prey_Result == True):
            return "Success"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
        prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Agent 8 Modified

In [ ]:
def agent_eight_modified(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"


       #Surveying a node
        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
            survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
        else:
            survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
        
        if result_prey == True:
            confidence.append(True)
        else:
            confidence.append(False)
        if result_pred == True:
            confidence_pred.append(True)
        else:
            confidence_pred.append(False)

        #Update probability at time t with new info for both prey and predator
        prob_dic = pred_prob_updates_Agent_modified(prob_dic,result_pred,survey_node)
        prob_dic_prey =  prey_prob_updates_Agent_modified(prob_dic_prey,result_prey,survey_node) # for prey

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])
        
        #agent moves
        agent_pos= agent_two_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"
        Prey_Result =  (agent_pos == prey_pos)
        if(Prey_Result == True):
            return "Success"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
        prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)

        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Modified AGent 7 without prob update

In [ ]:
def agent_seven_modified_without_update(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        #Surveying a node
        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
            survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
        else:
            survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
            

        #Update probability at time t with new info for both prey and predator
        prob_dic = pred_prob_updates_Agent(prob_dic,result_pred,survey_node)
        prob_dic_prey =  prey_prob_updates_Agent(prob_dic_prey,result_prey,survey_node) # for prey
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])
        
        #agent moves
        agent_pos= agent_one_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"
        Prey_Result =  (agent_pos == prey_pos)
        if(Prey_Result == True):
            return "Success"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
        prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Agent 8 modifies without prob updates

In [ ]:
def agent_eight_modified_without_update(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary

    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"


       #Surveying a node
        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
            survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
        else:
            survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
            result_pred = node_survey(survey_node,pred_pos)
            if result_pred == True:
                result_pred = random.choices([True, False],[0.9,0.1])[0]
            result_prey = node_survey(survey_node,prey_pos)
            if result_prey == True:
                result_prey = random.choices([True, False],[0.9,0.1])[0]
            
        #Update probability at time t with new info for both prey and predator
        prob_dic = pred_prob_updates_Agent(prob_dic,result_pred,survey_node)
        prob_dic_prey =  prey_prob_updates_Agent(prob_dic_prey,result_prey,survey_node) # for prey

        #getting the assumed prey postion
        pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
        pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])
        
        #agent moves
        agent_pos= agent_two_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"
        Prey_Result =  (agent_pos == prey_pos)
        if(Prey_Result == True):
            return "Success"

        #Update probability at time t with new info
        prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
        prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)

        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

Bonus 2

In [ ]:
def agent_bonus(agent_pos, prey_pos,pred_pos,maxsteps):

    prob_dic = pred_prob_initial(graph,agent_pos,pred_pos) #If we kno where the pred starts
    #prob_dic = prey_prob_initial(graph,agent_pos)

    prob_dic_prey = prey_prob_initial(graph, agent_pos) #prey probability dictionary
    steps = 0

    while agent_pos!=prey_pos and agent_pos!=pred_pos:
        steps = step_count(steps)
        if steps > maxsteps: #supposed to be maxsteps here
            return "Faliure2"

        if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0 and len([key for key, value in prob_dic_prey.items() if value >= 0.5]) == 0:
            #Surveying a node
            if len([key for key, value in prob_dic.items() if value >= 0.5]) == 0:
                survey_node = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
                result_pred = node_survey(survey_node,pred_pos)
                result_prey = node_survey(survey_node,prey_pos)
            else:
                survey_node = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())]) # surveying for prey
                result_pred = node_survey(survey_node,pred_pos)
                result_prey = node_survey(survey_node,prey_pos)
                

            #Update probability at time t with new info for both prey and predator
            prob_dic = pred_prob_updates_Agent(prob_dic,result_pred,survey_node)
            prob_dic_prey =  prey_prob_updates_Agent(prob_dic_prey,result_prey,survey_node) # for prey
            # print("Pred",sum(prob_dic.values()))
            # print("Prey",sum(prob_dic_prey.values()))

            #getting the assumed prey postion
            pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])

        else:
            pseudo_pred_pos = random.choice([key for key, value in prob_dic.items() if value == max(prob_dic.values())])
            pseudo_prey_pos = random.choice([key for key, value in prob_dic_prey.items() if value == max(prob_dic_prey.values())])    
            #agent moves
            agent_pos= agent_two_rules(agent_pos, pseudo_prey_pos, pseudo_pred_pos)
            Pred_Result =  (agent_pos == pred_pos)
            if(Pred_Result == True):
                return "Faliure1"
            Prey_Result =  (agent_pos == prey_pos)
            if(Prey_Result == True):
                return "Success"

            #Update probability at time t with new info
            prob_dic = pred_prob_updates_Agent(prob_dic,Pred_Result,agent_pos)
            prob_dic_prey = prey_prob_updates_Agent(prob_dic_prey, Prey_Result, agent_pos)
            # print("Pred",sum(prob_dic.values()))
            # print("Prey",sum(prob_dic_prey.values()))
       
        #moving prey
        prey_pos = move_prey(prey_pos)

        #pred_pos = move_predator(pred_pos, agent_pos) #* bec of the new writeup
        pred_pos = move_distracted_predator(pred_pos, agent_pos)
        Pred_Result =  (agent_pos == pred_pos)
        if(Pred_Result == True):
            return "Faliure1"

                #Update probability at time t+1
        prob_dic = pred_prob_updates_pred(prob_dic,graph, agent_pos)
        prob_dic_prey = prey_prob_updates_prey(prob_dic_prey, graph)
        # print("Pred",sum(prob_dic.values()))
        # print("Prey",sum(prob_dic_prey.values()))
        
        
    if agent_pos == prey_pos:
        return "Success"
    elif agent_pos == pred_pos:
        return "Faliure1"   

#Calculate Success Here

In [ ]:
def Success_rate(Agent, Prey, Pred):
    steps = 0
    steps_faliure1 = 0
    steps_faliure2 = 0
    for trials in range(1): #30
        result = agent_two_remix(Agent, Prey, Pred,maxsteps=5000)
        if result == "Success":
            steps = step_count(steps)
        elif result == "Faliure1":
            steps_faliure1 = step_count(steps_faliure1)
        elif result == "Faliure2":
            steps_faliure2 = step_count(steps_faliure2)
    return steps, steps_faliure1, steps_faliure2


In [ ]:

def calc_success_rate():
    s_list=[]
    f1_list =[]
    f2_list = []
    confidence_list=[]
    confidence_list_pred = []
    # Graph100 = gen_100_graphs()
    for runs in range(30):
        print("Initializing run: ", runs+1)
        Graph100 = gen_100_graphs()
        success_run = 0
        faliure1_run = 0
        faliure2_run = 0
        for num in Graph100:
            # print("Initializing graph: ",num+1)
            graph = Graph100[num]
            Positions = {"Agent": None,"Prey":None,"Pred":None}
            Positions = spawn(graph,Positions)
            success, faliure1, faliure2 = Success_rate(Positions["Agent"], Positions["Prey"], Positions["Pred"])
            success_run=success_run+success
            faliure1_run = faliure1_run + faliure1
            faliure2_run = faliure2_run+faliure2
        s_list.append(success_run)
        f1_list.append(faliure1_run)
        f2_list.append(faliure2_run)
        
        # confidence_list.append(confidence.count(True)/len(confidence)*100)
        # confidence_list_pred.append(confidence_pred.count(True)/len(confidence_pred)*100)
        # confidence.clear()
        # confidence_pred.clear()
    # print("Prey",np.mean(confidence_list))
    # print("Pred",np.mean(confidence_list_pred))
        
    # print(confidence_list)
    # print(confidence_list_pred)

    return s_list, f1_list,f2_list

s1, f1, f2 = calc_success_rate()

print("success rate: ",(np.sum(s1)/3000)*100)
print("faliure 1 rate: ",(np.sum(f1)/3000)*100)
print("faliure 2 rate: ",(np.sum(f2)/3000)*100)

